In [10]:
####################################################################################################################
####################################################################################################################
#PRE-PROCESSING
####################################################################################################################
####################################################################################################################


#Import pacakges
from datetime import datetime
import numpy as np
import pandas as pd
import pyomo.environ as pyo
import gc
import highspy
from pyomo.common.timing import TicTocTimer
import os


import sympy as sp
import statsmodels.api as sm


#import scripts
# import preprocessor_simple as prep
#import postprocessor as post

####################################################################################################################


In [15]:
##############################################################################################################
### UPDATE WITH JAMES' NEW LOAD FILE
##############################################################################################################

class residentialModule:
    prices = {}
    loads = {}
    hr_map = pd.DataFrame()
    baseYear = 2023
    

    def __init__(self):
        self.year = sp.Idx('year')
        self.reg = sp.Idx('region')
        self.fuel = sp.Idx('fuel')
        self.LastHYr, self.LastMYr, self.BaseYr = sp.symbols(('LastHYr','LastMYr','base'))

        self.income = sp.IndexedBase('Income')
        self.incomeIndex = sp.IndexedBase('IncomeIndex')
        self.i_elas = sp.IndexedBase('IncomeElasticity')
        self.i_lag = sp.IndexedBase('IncomeLagParameter')

        self.price = sp.IndexedBase('Price')
        self.priceIndex = sp.IndexedBase('PriceIndex')
        self.p_elas = sp.IndexedBase('PriceElasticity')
        self.p_lag = sp.IndexedBase('PriceLagParameter')
        
        self.trendGR = sp.IndexedBase('TrendGR')

        self.consumption = sp.IndexedBase('Consumption')

        self.incomeEQ = (self.incomeIndex[self.year-1,self.reg,self.fuel] ** self.i_lag[self.reg,self.fuel]) * (self.income[self.year,self.reg,self.fuel]/self.income[self.BaseYr,self.reg,self.fuel]) ** self.i_elas[self.reg,self.fuel]
        self.priceEQ = (self.priceIndex[self.year-1,self.reg,self.fuel] ** self.p_lag[self.reg,self.fuel]) * (self.price[self.year,self.reg,self.fuel]/self.price[self.BaseYr,self.reg,self.fuel]) ** self.p_elas[self.reg,self.fuel]
        self.growthEQ = 1 + ((self.year - self.LastHYr)/(self.LastMYr - self.LastHYr)) * (((1 + self.trendGR[self.reg,self.fuel]) ** (self.LastMYr - self.LastHYr)) - 1)

        self.QIndex = self.incomeEQ * self.priceEQ * self.growthEQ

        self.demand = self.consumption[self.BaseYr,self.reg,self.fuel] * self.QIndex

        self.lambdifiedDemand = sp.lambdify([self.incomeIndex[self.year-1,self.reg,self.fuel],
                                    self.i_lag[self.reg,self.fuel],
                                    self.income[self.year,self.reg,self.fuel],
                                    self.income[self.BaseYr,self.reg,self.fuel],
                                    self.i_elas[self.reg,self.fuel],
                                    self.priceIndex[self.year-1,self.reg,self.fuel],
                                    self.p_lag[self.reg,self.fuel],
                                    self.price[self.year,self.reg,self.fuel],
                                    self.price[self.BaseYr,self.reg,self.fuel],
                                    self.p_elas[self.reg,self.fuel],
                                    self.year,
                                    self.LastHYr,
                                    self.LastMYr,
                                    self.trendGR[self.reg,self.fuel],
                                    self.consumption[self.BaseYr,self.reg,self.fuel]],
                                    self.demand)
        
        if not self.prices:
            priceData = pd.read_excel('C:/Users/MLJ/Downloads/ECM Simple/input/cem_elec_prices.xlsx').set_index(['r','y','hr'],drop=False)
            loadData = pd.read_csv('C:/Users/MLJ/Downloads/ECM Simple/input/cem_inputs/Load.csv').set_index(['r','y','hr'],drop=False)
            # temploadData = temploadData.loc[temploadData.y == 2023].set_index(['y','hr'],drop=False)
            # cols = {f'r{i}':i for i in range(1,26)}
            # temploadData = temploadData.rename(columns=cols)
            # loadData = temploadData.loc[:,range(1,26)].stack().reset_index().rename(columns={'level_2':'r',0:'Load'}).set_index(['r','y','hr'],drop=False)
            # loadData = pd.DataFrame([(r,2023,hr) for r in range(1,26) for hr in range(1,8761)],columns=['r','y','hr']).set_index(['r','y','hr'],drop=False)
            # loadData['Load'] = loadData.apply(lambda row: temploadData.loc[(2023,row.hr),str(row.r)],axis=1)
            self.set_base_values(priceData,loadData)
            self.hr_map = self.temporal_mapping()
        else:
            self.demandF = self.create_demandF()
        
        pass

    def temporal_mapping(self):
        df1 = pd.read_csv('C:/Users/MLJ/Downloads/ECM Simple/input/sw_s_day.csv')
        df4 = df1.groupby(by=['Map_day'],as_index=False).count()
        df4 = df4.rename(columns={'Index_day':'Dayweights'}).drop(columns=['Map_s'])
        df1 = pd.merge(df1,df4,how='left',on=['Map_day'])
    
        df2 = pd.read_csv('C:/Users/MLJ/Downloads/ECM Simple/input/sw_hr.csv')
        df3 = df2.groupby(by=['Map_hr'],as_index=False).count()
        df3 = df3.rename(columns={'Index_hr':'Hr_weights'})
        df2 = pd.merge(df2,df3,how='left',on=['Map_hr'])
    
        df = pd.merge(df1,df2,how='cross')
        df['hr'] = df.index
        df['hr'] = df['hr'] + 1
        df['Map_hr'] = (df['Map_day'] - 1) * df['Map_hr'].max() + df['Map_hr']
        #df.to_csv('../input/temporal_map.csv',index=False)
        df.set_index('hr',inplace=True, drop=False)
        return df

    def create_demandF(self):
        demandF = lambda price, load, year, basePrice = 1, p_elas = -0.10, baseYear = self.prices['BasePrices'].y.unique(), baseIncome = 1, income = 1, i_elas = 1, priceIndex = 1, incomeIndex = 1, p_lag = 1, i_lag = 1, trend = 0 : \
            self.lambdifiedDemand(incomeIndex, i_lag, income, baseIncome, i_elas, priceIndex, p_lag, price, basePrice, p_elas, year, baseYear, 2050, trend, load)
        return demandF

    #Sets up base values
    def set_base_values(self, p, load):
        self.prices['BasePrices'] = p
        self.loads['BaseLoads'] = load.Load
        self.demandF = self.create_demandF()
        return
    
    def update_load(self, p):
        if type(p) == pyo.base.param.IndexedParam:
            newLoad = pd.DataFrame([(i[0],i[1],i[2],p[i]) for i in p],columns=['r','y','hr','Dual']).set_index(['r','y','hr'],drop=False)
        else:
            newLoad = p.copy()
        hours = newLoad.hr.unique()
        n = len(hours)
        #hard-coded for the 4 seasons from 8760 data
        if n == 4:
            hourMap = {}
            hourMap[1] = list(range(1,2161)) + list(range(8017,8761))
            hourMap[3] = range(2161,3625)
            hourMap[2] = range(3625,6553)
            hourMap[4] = range(6553,8017)
        elif n == 96:
            hourMap = {i:[] for i in hours}
            for h in range(1,8760):
                hourMap[self.hr_map.loc[h,'Map_hr']].append(h)
        # for h in hours:
        #     hourMap[h] = range(int(((h-1)*8760/n)+1),int((h*8760/n)+1))
        #newLoad['BasePrice'] = newLoad.apply(lambda row: sum(self.prices['BasePrices'].loc[(row.r,2023,hr),'Dual'] for hr in hourMap[row.hr])/(len(hourMap[row.hr])),axis=1)
        newLoad['Load'] = newLoad.apply(lambda row: self.demandF(p,self.loads['BaseLoads'].loc[(row.r,2023,row.hr)],row.y,row.BasePrice)[0],axis=1)
        return newLoad.Load
    
    #Creates a block that can be given to another pyomo model
    #The constraint is essentially just updating the load parameter
    
    #Creates a block that can be given to another pyomo model
    #The constraint is essentially just updating the load parameter
    def make_block(self, prices, pricesindex):
        loadIndex = []
        #for i in prices.index:
        for reg in range(7,9):
            for hour in range(1,96): 
                newr = int(reg)
                newhr = int(hour)
                loadIndex.append((newr,2023,newhr))
            #loadIndex.append((i[0],2023,i[2]))
        mod = pyo.ConcreteModel()
        mod.block = pyo.Block()

        #mod.block.price_set = pyo.Set(initialize=prices.index)
        mod.block.price_set = pyo.Set(initialize=pricesindex)
        mod.block.load_set = pyo.Set(initialize=loadIndex)
        
        
        #mod.block.prices = pyo.Param(mod.block.price_set, initialize=prices.Dual, mutable=True)
        #mod.block.prices = prices
        mod.block.base_load = pyo.Param(mod.block.load_set, initialize=self.loads['BaseLoads'].loc[loadIndex], mutable=True)
        updated_load = self.update_load(prices)

        mod.block.Load = pyo.Var(mod.block.price_set, within=pyo.NonNegativeReals)

        @mod.block.Constraint(mod.block.price_set)
        def create_load(block,r,y,hr):
            return block.Load[r,y,hr] == updated_load.loc[(r,y,hr)]
        
        return mod.block
        
        
    #This is a standalone pyomo model
    #The objective function doesn't have much meaning
    #The real purpose is to have the load parameter changed into a variable that gets updated by the new prices
    def make_pyomo_model(self, prices):
        loadIndex = []
        for i in prices.index:
            loadIndex.append((i[0],2023,i[2]))
        mod = pyo.ConcreteModel()

        mod.price_set = pyo.Set(initialize=prices.index)
        mod.load_set = pyo.Set(initialize=loadIndex)
        
        mod.prices = pyo.Param(mod.price_set, initialize=prices.Dual, mutable=True)
        mod.base_load = pyo.Param(mod.load_set, initialize=self.loads['BaseLoads'].loc[loadIndex], mutable=True)
        updated_load = self.update_load(prices)
        
        mod.Load = pyo.Var(mod.price_set, within=pyo.NonNegativeReals)

        mod.obj = pyo.Objective(rule = sum(mod.Load[r,y,hr] for [r,y,hr] in mod.price_set))

        @mod.Constraint(mod.price_set)
        def create_new_load(mod,r,y,hr):
            return mod.Load[r,y,hr] == updated_load.loc[(r,y,hr)]
        
        return mod

In [12]:
model = residentialModule()
model.demandF

model2 = residentialModule()
print(model2.demandF(1,1,1))

[1.]


In [16]:
model = residentialModule()
import preprocessor_simple as prep
############################################
# RUN NEW RESIDENTIAL MODEL BY ITSELF
############################################
model3 = pyo.ConcreteModel(name='MultiBlock')
model3.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

test_years = list(pd.read_csv('C:/Users/MLJ/Downloads/ECM/input/sw_year.csv').dropna()['year'])
test_regions = list(pd.read_csv('C:/Users/MLJ/Downloads/ECM/input/sw_reg.csv').dropna()['region'])

all_frames, setin = prep.preprocessor(prep.Sets(test_years,test_regions),'C:/Users/MLJ/Downloads/ECM Simple/input/temp/')


#initialize sets

model3.SupplyPriceSet = pyo.Set(initialize = all_frames['SupplyPrice'].index)
model3.SupplyPrice = pyo.Param(model3.SupplyPriceSet, initialize = all_frames['SupplyPrice'], default = 0, mutable = True)

newPrices = model3.SupplyPrice
#newPrices = pd.DataFrame([(1,2024,1,11.59),(1,2024,2,12.25),(1,2024,3,14.81),(1,2024,4,11.53)],columns=['r','y','hr','Dual']).set_index(['r','y','hr'],drop=False)
model_res = residentialModule()
newBlock = model_res.make_block(model3.SupplyPrice, model3.SupplyPriceSet)
newBlock.pprint()

KeyError: (1.0, 2023, 1.0)